# **Normalized Difference Built-Up Index (NDBI) Dubai**

# Urban Evolution of Dubai - A Comparative Remote Sensing Analysis from 1995 to 2024

**Erik Ashkinadze (erik.ashkinadze@ruhr-uni-bochum.de)**

**Devon Klör (devon.kloer@ruhr-uni-bochum.de)**

**Course:** Geographic Information Systems (GIS I): Databases and Programming

**Professor:** Jun.-Prof. Dr. Andreas Rienow

**References:** Zha, Y.; Gao, J.; Ni, S. (2003): Use of Normalized Difference Built-Up Index in Automatically Mapping Urban Areas from TM Imagery. In: International Journal of Remote Sensing, 24: 583-594.
https://doi.org/10.1080/01431160304987

**Repository:** GIS 1 Course "03_Create_landsat_timelapse_byQuishengWu"

In [ ]:
import ee
import geemap

# Authenticate and Initialize Earth Engine with the project ID
ee.Authenticate()
ee.Initialize(project='ee-dkloer01')

# Shapefile einladen (Pfad zur Datei anpassen)
shapefile_path = "./Dubai_Shapes/Dubai.shp"
dubai = geemap.shp_to_ee(shapefile_path)

# Loading the Image Collections from Landsat 5, 7 and 8
# Landsat 5 for 1995 (TM Sensor)
ls_ic_1995 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate('1995-01-01', '1995-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 7 for 2005 (TM Sensor)
ls_ic_2005 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterDate('2005-01-01', '2005-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 8 for 2015 (OLI Sensor)
ls_ic_2015 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2015-01-01', '2015-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 8 for 2025 (OLI Sensor)
ls_ic_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(dubai) \
    .median()

# Calculation of the Normalized Difference Vegetation Index (NDVI)
# Using band 4 (NIR) and band 3 (Red) for historic analysis with Landsat 5 & 7
# Using band 5 (NIR) and band 4 (Red) for current analysis with Landsat 8
def calculate_ndvi(image, year):
    if year <= 2005:
        # Landsat 5 & 7
        nir = image.select('SR_B4')
        red = image.select('SR_B3')
    else:
        # Landsat 8
        nir = image.select('SR_B5')
        red = image.select('SR_B4')

    # NDVI formula (orientated on Huang et al. 2020)
    numerator = (nir.subtract(red))
    denominator = (nir.add(red))
    ndvi = numerator.divide(denominator)
    return ndvi.addBands(ndvi.rename('NDVI'))

# Calculate NDVI for each year
ndvi_bands_1995 = calculate_ndvi(ls_ic_1995, 1995)
ndvi_1995 = ndvi_bands_1995.select('NDVI')

ndvi_bands_2005 = calculate_ndvi(ls_ic_2005, 2005)
ndvi_2005 = ndvi_bands_2005.select('NDVI')

ndvi_bands_2015 = calculate_ndvi(ls_ic_2015, 2015)
ndvi_2015 = ndvi_bands_2015.select('NDVI')

ndvi_bands_2024 = calculate_ndvi(ls_ic_2024, 2024)
ndvi_2024 = ndvi_bands_2024.select('NDVI')


# Calculation of the Normalized Difference Built-Up Index (NDBI)
# Using band 5 (SWIR1), band 4 (NIR) and band 1 (Blue) for historic analysis with Landsat 5 & 7
# Using band 6 (SWIR1), band 5 (NIR) and band 2 (Blue) for current analysis with Landsat 8
def calculate_ndbi(image, year):
    if year <= 2005:
        # Landsat 5 & 7
        swir1 = image.select('SR_B5')
        nir = image.select('SR_B4')
    else:
        # Landsat 8
        swir1 = image.select('SR_B6')
        nir = image.select('SR_B5')

    # NDBI formula (orientated on Zha et al. 2003)
    numerator = (swir1.subtract(nir))
    denominator = (swir1.add(nir))
    ndbi = numerator.divide(denominator)
    return ndbi.addBands(ndbi.rename('NDBI'))

# Calculate NDVI for each year
ndbi_bands_1995 = calculate_ndbi(ls_ic_1995, 1995)
ndbi_1995 = ndbi_bands_1995.select('NDBI')

ndbi_bands_2005 = calculate_ndbi(ls_ic_2005, 2005)
ndbi_2005 = ndbi_bands_2005.select('NDBI')

ndbi_bands_2015 = calculate_ndbi(ls_ic_2015, 2015)
ndbi_2015 = ndbi_bands_2015.select('NDBI')

ndbi_bands_2024 = calculate_ndbi(ls_ic_2024, 2024)
ndbi_2024 = ndbi_bands_2024.select('NDBI')

# NDBI Change Detection (Difference between 1995 und 2024)
ndbi_change = ndbi_2024.subtract(ndbi_1995).rename('NDBI Change')

# Calculation of Built Areas by subtracting NDVI from NDBI
ba_1995 = ndbi_1995.subtract(ndvi_1995).rename('BA').clip(dubai)
ba_2005 = ndbi_2005.subtract(ndvi_2005).rename('BA').clip(dubai)
ba_2015 = ndbi_2015.subtract(ndvi_2015).rename('BA').clip(dubai)
ba_2024 = ndbi_2024.subtract(ndvi_2024).rename('BA').clip(dubai)

# NDBI Change Detection (Difference between 1995 und 2024)
ba_change = ba_2024.subtract(ba_1995).rename('BA Change').clip(dubai)


# A color palette for visualizing NDBI (Normalized Difference Built-up Index) values.
ndbi_palette = [
    '#ff0501',  
    '#ffffef',  
    '#1a05a9'  
]

# A color palette for visualizing the change in NDBI values over time
ndbi_change_palette = [
    'red',  
    'white',  
    'blue'   
]

# These visualization settings define how the NDBI values are mapped to the color palette
ndbi_vis = {
    'min': -0.3, # The minimum NDBI value to be displayed in the visualization. Values below this will be colored with the lowest end of the palette.
    'max': 0.3, # The maximum NDBI change value to be displayed. A positive value indicates an increase in NDBI.
    'palette': ndbi_palette
}

# These visualization settings define how the NDBI change values are visualized
ndbi_change_vis = {
    'min': -0.3,
    'max': 0.3,
    'palette': ndbi_change_palette
}

# Create an interactive Map
map = geemap.Map(center=[25.07, 55.18], zoom=10)

# Add the Study Area as a Maplayer
map.addLayer(dubai.style(**{'color': 'black', 'width': 2}), {}, 'Study Area')

# Add the NDVI Layer to the Map
map.addLayer(ndbi_1995.clip(dubai), ndbi_vis, 'NDBI 1995')
map.addLayer(ndbi_2005.clip(dubai), ndbi_vis, 'NDBI 2005')
map.addLayer(ndbi_2015.clip(dubai), ndbi_vis, 'NDBI 2015')
map.addLayer(ndbi_2024.clip(dubai), ndbi_vis, 'NDBI 2024')
map.addLayer(ndbi_change.clip(dubai), ndbi_change_vis, 'NDBI Change')

# Add the Built Areas Layer to the Map
map.addLayer(ba_1995.clip(dubai), ndbi_vis, 'BA 1995')
map.addLayer(ba_2005.clip(dubai), ndbi_vis, 'BA 2005')
map.addLayer(ba_2015.clip(dubai), ndbi_vis, 'BA 2015')
map.addLayer(ba_2024.clip(dubai), ndbi_vis, 'BA 2024')
map.addLayer(ba_change.clip(dubai), ndbi_change_vis, 'BA Change')

# Display the Map
map


Map(center=[25.07, 55.18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
years = [1995, 2005, 2015, 2024]
# It takes a list of 'ee.Image' objects as input and combines them into a single 'ee.ImageCollection' object.
ba_collection = ee.ImageCollection([ba_1995, ba_2005, ba_2015, ba_2024])

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'BA', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'BA', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '1'}}, {'type': 'Image', 'bands': [{'id': 'BA', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '2'}}, {'type': 'Image', 'bands': [{'id': 'BA', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '3'}}]}


In [ ]:
ba_change_ = ee.ImageCollection([ba_change])

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'BA Change', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '0'}}]}


In [ ]:
# A color palette for visualizing Built-Up area values.
ba_palette = [
    '#ff0501',  
    '#ffffef',  
    '#1a05a9'  
]

# These visualization settings define how the BA values are mapped to the color palette
ba_vis = {
    'min': -0.3, 
    'max': 0.3, 
    'palette': ba_palette
}


In [ ]:
# This dictionary 'video_args_ba' contains the necessary parameters to generate a time-lapse video of the BA data for Dubai.
video_args_ba = {
    'dimensions': 768,
    'region': dubai.geometry().bounds(),
    'framesPerSecond': 2,
    'min': ba_vis['min'],
    'max': ba_vis['max'],
    'palette': ba_vis['palette'],
    'format': 'gif'
}

In [ ]:
# specifies the output file path and name for the generated GIF video
out_gif_ba = './ba_selected_years.gif'
geemap.download_ee_video(ba_collection, video_args_ba, out_gif_ba)

# This function takes an existing GIF video and adds text overlays to each frame.
geemap.add_text_to_gif(out_gif_ba, './ba_dubai_text.gif', text_sequence=years,
                       xy=('3%', '5%'), font_size=30, font_color='white', add_progress_bar=False, duration=500)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\dkloe\Downloads\GIS1_Codes\ba_selected_years.gif


In [ ]:
out_gif_ba_change = './ba_change.gif'
geemap.download_ee_video(ba_change_, video_args_ba, out_gif_ba_change)

geemap.add_text_to_gif(out_gif_ba_change, './ba_dubai_change.gif', text_sequence='Change Detection NDBI',
                       xy=('45%', '5%'), font_size=30, font_color='white', add_progress_bar=False, duration=500)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\dkloe\Downloads\GIS1_Codes\ba_change.gif


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

ba_palette = [
    '#1a05a9',
    '#ffffef', # Keine Veränderung (Gelb)
    '#ff0501'
]

# Creating a custom colormap from the list of colors
cmap_ba = LinearSegmentedColormap.from_list("my_ba_cmap", ba_palette)

fig, ax = plt.subplots(figsize=(6, 1), facecolor='black') # This line creates a Matplotlib figure and an axes object.
gradient = np.linspace(-1, 1, 256).reshape(1, -1) # This line creates a 1D NumPy array named 'gradient' containing 256 evenly spaced values between -1 and 1 
ax.imshow(gradient, aspect='auto', cmap=cmap_ba) 

# Re-enable axes
ax.set_axis_on()

# Set ticks and labels
ax.set_xticks([0, 255])  # Positions for the Ticks 
ax.set_xticklabels(['-1', '1'], color='white') # Labels for the Ticks


ax.set_yticks([])

# Remove top, right, and left axis spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('white') # This line sets the color of the bottom axis spine to white

# This line sets the color of the x-axis tick marks to white
ax.tick_params(axis='x', colors='white')

plt.savefig("legend_NDBI.png", bbox_inches='tight', pad_inches=0)
plt.close()

In [ ]:
ba_change_palette = [
    '#1a05a9',
    '#ffffef', 
    '#ff0501'
]

# Creating a custom colormap from the list of colors
cmap_ba_change = LinearSegmentedColormap.from_list("my_ba_change_cmap", ba_change_palette)

fig, ax = plt.subplots(figsize=(6, 1), facecolor='black') # This line creates a Matplotlib figure and an axes object.
gradient = np.linspace(-1, 1, 256).reshape(1, -1) # This line creates a 1D NumPy array named 'gradient' containing 256 evenly spaced values between -1 and 1 
ax.imshow(gradient, aspect='auto', cmap=cmap_ba_change) 

# Re-enable axes
ax.set_axis_on()

# Set ticks and labels
ax.set_xticks([0, 255])  # Positions for the Ticks 
ax.set_xticklabels(['decrease', 'increase'], color='white') # Labels for the Ticks

ax.set_yticks([])

# Remove top, right, and left axis spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('white') # This line sets the color of the bottom axis spine to white

# This line sets the color of the x-axis tick marks to white
ax.tick_params(axis='x', colors='white')

plt.savefig("legend_NDBI_Change.png", bbox_inches='tight', pad_inches=0)
plt.close()